In [0]:
!pip install langchain openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/730.2 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 23.9 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

Agent Use Cases:
1. Lodging - Airbnb
2. Lodging - Booking
3. Lodging - Google
4. Itinerary
5. Location to Location

Need to determine lodging first or itinerary first
Location to location will always come last

In [0]:
from openai import OpenAI
import os

# How to get your Databricks token: https://docs.databricks.com/en/dev-tools/auth/pat.html
# DATABRICKS_TOKEN = os.environ.get('DATABRICKS_TOKEN')
# Alternatively in a Databricks notebook you can use this:
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

client = OpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url="https://dbc-853dca7e-4798.cloud.databricks.com/serving-endpoints"
)

response = client.chat.completions.create(
    model="Maverick-Hackathon",
    messages=[
        {
            "role": "user",
            "content": "What is an LLM agent?"
        }
    ]
)

print(response.choices[0].message.content)

An LLM agent is a software entity that utilizes a Large Language Model (LLM) to perform various tasks. To understand what an LLM agent is, let's break down the components:

1. **Large Language Model (LLM)**: LLMs are a type of artificial intelligence (AI) model designed to process and generate human-like language. These models are trained on vast amounts of text data, enabling them to understand and respond to a wide range of inputs, from simple queries to complex tasks.

2. **Agent**: In the context of computing and AI, an agent refers to a program or system that can perceive its environment, make decisions based on its perceptions, and act upon its environment to achieve certain goals. Agents can be simple or complex, ranging from a basic software bot that performs a routine task to sophisticated systems that can learn and adapt.

When we talk about an **LLM agent**, we're referring to a software agent that leverages the capabilities of a Large Language Model. This means the agent ca

In [0]:
from langchain.agents import initialize_agent, AgentType
from langchain.chains import SequentialChain

In [0]:
def call_llm(system_prompt, inputs):
    data = {
        "model": "Maverick-Hackathon",
        "messages": [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": str(inputs)
            }
        ]
    }
    response = client.chat.completions.create(**data)
    return response.choices[0].message.content

In [0]:
#def call_llm(system_prompt, inputs):
#    data = {
#        "inputs": [
#            {
#                "input": f"{system_prompt}\nInputs: {inputs}"
#            }
#        ]
#    }
#    result = query_model_endpoint(data)
#    return result

def generate_itinerary(accessibility_needs, destination, dates):
    system_prompt = (
        f"You are a helpful travel agent. Your task is to generate a travel itinerary for a user with specific accessibility needs. "
        f"The user may not have specific plans in mind. The user is arriving to {destination}. "
        f"Date Context {dates if len(dates) > 0 else 'No dates provided'}. Accessibility Needs: {accessibility_needs}."
    )
    inputs = {
        "accessibility_needs": accessibility_needs,
        "destination": destination,
        "dates": dates
    }
    return call_llm(system_prompt, inputs)

def find_lodging(accessibility_needs, destination, dates):
    system_prompt = (
        "You're a helpful travel agent. Your task is to find lodging options in a specific destination for a given date range that meet the user's accessibility needs. "
        "Ensure that the lodging has the accomodations necessary for the user. If you cannot meet those requirements, then return 'No lodging with accomodations found'."
    )
    inputs = {
        "accessibility_needs": accessibility_needs,
        "destination": destination,
        "dates": dates
    }
    return call_llm(system_prompt, inputs)

def create_booking(lodging_info, accessibility_needs, user_info):
    system_prompt = (
        f"Given the lodging information and trip context. Pre-fill the form for the user. "
        f"Context: {lodging_info}; User Information: {user_info}."
    )
    inputs = {
        "lodging_info": lodging_info,
        "accessibility_needs": accessibility_needs,
        "user_info": user_info
    }
    return call_llm(system_prompt, inputs)

def generate_directions(origin, destination, accessibility_needs, date):
    system_prompt = (
        f"Given that the user is going to {destination} on {date}, generate detailed, accessible directions from their home to the destination. "
        f"Allow ample time and include step-by-step accessible options for the following needs: {accessibility_needs}."
    )
    inputs = {
        "origin": origin,
        "destination": destination,
        "accessibility_needs": accessibility_needs,
        "date": date
    }
    return call_llm(system_prompt, inputs)

def generate_overall_plan(accessibility_needs, destination, dates, user_info):
    system_prompt = (
        f"Given the knowledge of the previous steps, combine the following into a comprehensive, accessible travel plan. "
        f"Ensure accessibility is prioritized throughout. Be as detailed as possible. Provide each day as a separate response, so that way you can maximize the information provided to the user. "
        f"Provide the location or lodging website URL. Lodging: {{lodging}}, Booking: {{booking}}, Directions: {{directions}}."
    )
    inputs = {
        "accessibility_needs": accessibility_needs,
        "destination": destination,
        "dates": dates,
        "user_info": user_info
    }
    return call_llm(system_prompt, inputs)

In [0]:
class Step:
    def __init__(self, name, function, inputs, system_prompt):
        self.name = name
        self.function = function
        self.inputs = inputs
        self.system_prompt = system_prompt
        self.output = None
        self.input_keys = tuple(inputs)

    def run(self):
        # Implement the logic to execute the function with inputs
        self.output = self.function(*self.inputs)
        return self.output
    
class TravelAgent:
    def __init__(self, accessibility_needs, destination, dates, user_info):
        self.accessibility_needs = accessibility_needs
        self.destination = destination
        self.dates = dates
        self.user_info = user_info
        self.lodging_info = None

    def generate_itinerary_step(self):
        return Step(
            name="Generate Itinerary",
            function=generate_itinerary,
            inputs=[self.accessibility_needs, self.destination, self.dates],
            system_prompt = "You are a helpful travel agent. Your task is to generate a travel itinerary for a user with specific accessibility needs. The user may not have specific plans in mind. The user is arriving to {self.destination}. Date Context {self.dates if len(self.dates) > 0 else 'No dates provided'}. Accessibility Needs: {self.accessibility_needs}."
        )

    def find_lodging_step(self):
        return Step(
            name="Find Lodging",
            function=find_lodging,
            inputs=[self.accessibility_needs, self.destination, self.dates],
            system_prompt = "You're a helpful travel agent. Your task is to find lodging options in a specific destination for a given date range that meet the user's accessibility needs. Ensure that the lodging has the accomodations necessary for the user. If you cannot meet those requirements, then return 'No lodging with accomodations found'."
        )

    def create_booking_step(self):
        return Step(
            name="Create Booking",
            function=create_booking,
            inputs=[self.lodging_info, self.accessibility_needs, self.user_info],
            system_prompt = "Given the lodging information and trip context. Pre-fill the from for the user. Context: {self.lodging_info}; User Information: {self.user_info}."
        )

    def generate_directions_step(self):
        return Step(
            name="Generate Directions",
            function=generate_directions,
            inputs=["Home", self.destination, self.accessibility_needs, self.dates[0]],
            system_prompt = "Given that the user is going to {self.destination} on {self.dates[0]}, generate detailed, accessible directions from their home to the destination. Allow ample time and include step-by-step accessible options for the following needs: {self.accessibility_needs}."
        )

    def generate_overall_plan_step(self):
        return Step(
            name="Generate Overall Plan",
            function=generate_overall_plan,
            inputs=[self.accessibility_needs, self.destination, self.dates, self.user_info],
            system_prompt = "Given the knowledge of the previous steps, combine the following into a comprehensive, accessible travel plan. Ensure accessibility is prioritized throughout. Be as detailed as possible. Provide each day as a separate response, so that way you can maximize the information provided to the user. Provide the location or lodging website URL. Lodging: {self.find_lodging_step().output}, Booking: {self.create_booking_step().output}, Directions: {self.generate_directions_step().output}."
        )

    def specify_accessibility_needs():
        accessibility_needs = input("Please specify your accessibility needs (e.g., wheelchair access, visual aids): ")
        return accessibility_needs

    def update_user_info():
        first_name = input("Please enter your first name: ")
        last_name = input("Please enter your last name: ")
        date_of_birth = input("Please enter your date of birth (YYYY-MM-DD): ")
        email = input("Please enter your email: ")
        phone_number = input("Please enter your phone number: ")
        user_info = {
            "first_name": first_name,
            "last_name": last_name,
            "date_of_birth": date_of_birth,
            "email": email,
            "phone_number": phone_number
        }
        return user_info

    def create_chain(self):
        itinerary_step = self.generate_itinerary_step()
        itinerary_step.run()
        
        lodging_step = self.find_lodging_step()
        self.lodging_info = lodging_step.run()  # Update lodging_info attribute
        
        booking_step = self.create_booking_step()
        booking_output = booking_step.run()
        
        overall_plan_output = []
        for date in self.dates:
            directions_step = self.generate_directions_step(date)
            directions_output = directions_step.run()
            
            overall_plan_step = self.generate_overall_plan_step(date)
            day_plan = overall_plan_step.run()
            
            # Ensure no duplicate activities
            if day_plan not in self.activities:
                self.activities.add(day_plan)
                overall_plan_output.append(day_plan)
        
        return overall_plan_output



In [0]:
# Example usage
accessibility_needs = "wheelchair access, visual aids"
destination = "Paris"
dates = ["2025-06-10", "2025-06-20"]
user_info = {"name": "John Doe", "contact": "john.doe@example.com"}

travel_agent = TravelAgent(accessibility_needs, destination, dates, user_info)
travel_chain = travel_agent.create_chain()
#result = travel_chain.run()
display(travel_chain)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-8300164410613005>, line 8
      5 user_info = {"name": "John Doe", "contact": "john.doe@example.com"}
      7 travel_agent = TravelAgent(accessibility_needs, destination, dates, user_info)
----> 8 travel_chain = travel_agent.create_chain()
      9 #result = travel_chain.run()
     10 display(travel_chain)

File <command-8300164410612857>, line 94, in TravelAgent.create_chain(self)
     92 overall_plan_output = []
     93 for date in self.dates:
---> 94     directions_step = self.generate_directions_step(date)
     95     directions_output = directions_step.run()
     97     overall_plan_step = self.generate_overall_plan_step(date)

TypeError: TravelAgent.generate_directions_step() takes 1 positional argument but 2 were given

In [0]:
class TravelAgent:
    def __init__(self, accessibility_needs, destination, dates, user_info):
        self.accessibility_needs = accessibility_needs
        self.destination = destination
        self.dates = dates
        self.user_info = user_info
        self.lodging_info = None

    def generate_itinerary_step(self):
        return Step(
            name="Generate Itinerary",
            function=generate_itinerary,
            inputs=[self.accessibility_needs, self.destination, self.dates],
            system_prompt = "You are a helpful travel agent. Your task is to generate a travel itinerary for a user with specific accessibility needs. The user may not have specific plans in mind, so prioritize convenience and accessibility while viewing the key attractions of each destination. The user is arriving to {self.destination}. Date Context {self.dates if len(self.dates) > 0 else 'No dates provided'}. Accessibility Needs: {self.accessibility_needs}."
        )

    def find_lodging_step(self):
        return Step(
            name="Find Lodging",
            function=find_lodging,
            inputs=[self.accessibility_needs, self.destination, self.dates],
            system_prompt = "You're a helpful travel agent. Your task is to find lodging options in a specific destination for a given date range that meet the user's accessibility needs. Ensure that the lodging has the accomodations necessary for the user, if you cannot meet those requirements, then return 'No lodging with accomodations found'."
        )

    def create_booking_step(self):
        return Step(
            name="Create Booking",
            function=create_booking,
            inputs=[self.lodging_info, self.accessibility_needs, self.user_info],
            system_prompt = "Given the lodging information and trip context. Pre-fill the from for the user. Context: {self.lodging_info}; User Information: {self.user_info}."
        )

    def generate_directions_step(self, date):
        return Step(
            name="Generate Directions",
            function=generate_directions,
            inputs=["Home", self.destination, self.accessibility_needs, date],
            system_prompt = "Given that the user is going to {self.destination} on {date}, generate detailed, accessible directions from their home to the destination. Allow ample time and include step-by-step accessible options for the following needs: {self.accessibility_needs}."
        )

    def generate_overall_plan_step(self, date):
        return Step(
            name="Generate Overall Plan",
            function=generate_overall_plan,
            inputs=[self.accessibility_needs, self.destination, [date], self.user_info],
            system_prompt = "Given the knowledge of the previous steps, combine the following into a comprehensive, accessible travel plan. Ensure accessibility is prioritized throughout. Be as detailed as possible. Provide each date as a separate response, so that way you can maximize the information provided to the user. Provide the location or lodging website URL. Lodging: {self.find_lodging_step().output}, Booking: {self.create_booking_step().output}, Directions: {self.generate_directions_step(date).output}."
        )

    def create_chain(self):
        itinerary_step = self.generate_itinerary_step()
        itinerary_step.run()
        
        lodging_step = self.find_lodging_step()
        self.lodging_info = lodging_step.run()  # Update lodging_info attribute
        
        booking_step = self.create_booking_step()
        booking_output = booking_step.run()
        
        overall_plan_output = []
        for date in self.dates:
            directions_step = self.generate_directions_step(date)
            directions_output = directions_step.run()
            
            overall_plan_step = self.generate_overall_plan_step(date)
            day_plan = overall_plan_step.run()
            
            overall_plan_output.append(day_plan)
        
        return overall_plan_output


_1
"**Accessible Paris Travel Plan for John Doe** ### Day 1: 2025-06-10 #### Lodging: You will be staying at the **Hotel Le Walt** (https://www.hotel-lewalt.com/), a wheelchair-accessible hotel located in the heart of Paris. The hotel has adapted rooms with roll-in showers and visual aids for guests with disabilities. - **Booking:** You can book your stay through the hotel's website or via a third-party booking site like Booking.com (https://www.booking.com/hotel/fr/hotel-le-walt.html). Ensure to specify your accessibility needs during the booking process. - **Directions:** The hotel is located at 8 Rue de la Huchette, 75005 Paris. To get there from Paris Charles de Gaulle Airport, you can take a taxi or the RER B train towards Robinson/Saint-Rémy-lès-Chevreuse, get off at Saint-Michel - Notre-Dame, then take a short taxi ride or walk to the hotel. The hotel is within walking distance to many Parisian landmarks. #### Itinerary: 1. **Morning:** Start your day at the **Notre-Dame Cathedral** (https://www.notredamedeparis.fr/en/), a short walk from your hotel. The cathedral is wheelchair accessible via the rue du Cloître-Notre-Dame entrance. Visual aids and audio descriptions are available upon request. 2. **Lunch:** Head to **Le Comptoir du Relais** (https://www.comptoir-relais.com/), a wheelchair-accessible bistro near the cathedral. They can accommodate guests with various dietary needs; it's advisable to call ahead to discuss your requirements. 3. **Afternoon:** Visit the **Louvre Museum** (https://www.louvre.fr/en/accessibility), which is fully wheelchair accessible with ramps and elevators. The museum offers adapted restrooms, audio descriptions, and tactile models for visitors with visual impairments. You can rent a wheelchair or borrow one free of charge with a deposit. 4. **Dinner:** Dine at **Le Grand Vefour** (https://www.restaurant-legrandvefour.com/), a historic restaurant that is wheelchair accessible. They offer a sophisticated dining experience and can cater to various dietary needs. 5. **Evening:** End your day with a scenic **Seine River Cruise** (https://www.vedettesdeparis.com/en/). Many cruise operators offer wheelchair-accessible boats. Some also provide visual aids like subtitles or audio descriptions. #### Accessibility Tips: - Ensure to verify the accessibility features of each location before heading out, as changes can occur. - Consider purchasing a **Paris Visite** ticket for unlimited travel on public transport and some attractions. - For any assistance or to confirm accessibility features, contact the venues directly or reach out to the Paris Tourist Office (https://www.parisinfo.com/). For further assistance or to make adjustments to this itinerary, please contact John Doe at john.doe@example.com."


In [0]:
class Step:
    def __init__(self, name, function, inputs, system_prompt):
        self.name = name
        self.function = function
        self.inputs = inputs
        self.system_prompt = system_prompt
        self.output = None
        self.input_keys = tuple(inputs)

    def run(self):
        # Implement the logic to execute the function with inputs
        self.output = self.function(*self.inputs)
        return self.output

class TravelAgent:
    def __init__(self, accessibility_needs, destination, dates, user_info):
        self.accessibility_needs = accessibility_needs
        self.destination = destination
        self.dates = dates
        self.user_info = user_info
        self.lodging_info = None
        self.itinerary = []

    def generate_itinerary_step(self, date):
        return Step(
            name="Generate Itinerary",
            function=generate_itinerary,
            inputs=[self.accessibility_needs, self.destination, [date]],
            system_prompt = "You are a helpful travel agent. Your task is to generate a travel itinerary for a user with specific accessibility needs for the date {date}. The user may not have specific plans in mind, so prioritize convenience and accessibility while viewing the key attractions of each destination. The user is arriving to {self.destination}. Date Context {date}. Accessibility Needs: {self.accessibility_needs}."
        )

    def find_lodging_step(self):
        return Step(
            name="Find Lodging",
            function=find_lodging,
            inputs=[self.accessibility_needs, self.destination, self.dates],
            system_prompt = "You're a helpful travel agent. Your task is to find lodging options in a specific destination for a given date range that meet the user's accessibility needs. Ensure that the lodging has the accommodations necessary for the user, if you cannot meet those requirements, then return 'No lodging with accommodations found'."
        )

    def create_booking_step(self):
        return Step(
            name="Create Booking",
            function=create_booking,
            inputs=[self.lodging_info, self.accessibility_needs, self.user_info],
            system_prompt = "Given the lodging information and trip context. Pre-fill the form for the user. Context: {self.lodging_info}; User Information: {self.user_info}."
        )

    def generate_directions_step(self, start_location, end_location, date):
        return Step(
            name="Generate Directions",
            function=generate_directions,
            inputs=[start_location, end_location, self.accessibility_needs, date],
            system_prompt = "Given that the user is going to {end_location} on {date}, generate detailed, accessible directions from {start_location} to the destination. Allow ample time and include step-by-step accessible options for the following needs: {self.accessibility_needs}."
        )

    def generate_overall_plan_step(self, date, itinerary):
        return Step(
            name="Generate Overall Plan",
            function=generate_overall_plan,
            inputs=[self.accessibility_needs, self.destination, [date], self.user_info, itinerary],
            system_prompt = "Given the knowledge of the previous steps, combine the following into a comprehensive, accessible travel plan. Ensure accessibility is prioritized throughout. Be as detailed as possible. Provide each date as a separate response, so that way you can maximize the information provided to the user. Provide the location or lodging website URL. Lodging: {self.find_lodging_step().output}, Booking: {self.create_booking_step().output}, Directions: {self.generate_directions_step(itinerary[i-1] if i > 0 else 'Home', itinerary[i], date).output for i in range(len(itinerary))}."
        )

    def create_chain(self):
        lodging_step = self.find_lodging_step()
        self.lodging_info = lodging_step.run()  # Update lodging_info attribute
        
        booking_step = self.create_booking_step()
        booking_output = booking_step.run()
        
        overall_plan_output = []
        for date in self.dates:
            itinerary_step = self.generate_itinerary_step(date)
            itinerary = itinerary_step.run()
            self.itinerary.append(itinerary)
            
            day_plan = []
            for i in range(len(itinerary)):
                start_location = self.lodging_info if i == 0 else itinerary[i-1]
                end_location = itinerary[i]
                directions_step = self.generate_directions_step(start_location, end_location, date)
                directions_output = directions_step.run()
                day_plan.append(directions_output)
            
            overall_plan_step = self.generate_overall_plan_step(date, itinerary)
            day_plan.append(overall_plan_step.run())
            
            overall_plan_output.append(day_plan)
        
        return overall_plan_output

In [0]:
class TravelAgent:
    def __init__(self, accessibility_needs, destination, dates, user_info):
        self.accessibility_needs = accessibility_needs
        self.destination = destination
        self.dates = dates
        self.user_info = user_info
        self.lodging_info = None
        self.high_level_itineraries = []

    def generate_itinerary_step(self, date):
        return Step(
            name="Generate Itinerary",
            function=generate_itinerary,
            inputs=[self.accessibility_needs, self.destination, [date]],
            system_prompt = "You are a helpful travel agent. Your task is to generate a high-level travel itinerary for a user with specific accessibility needs for the date {date}. The user may not have specific plans in mind, so prioritize convenience and accessibility while viewing the key attractions of each destination. The user is arriving to {self.destination}. Date Context {date}. Accessibility Needs: {self.accessibility_needs}."
        )

    def find_lodging_step(self):
        return Step(
            name="Find Lodging",
            function=find_lodging,
            inputs=[self.accessibility_needs, self.destination, self.dates],
            system_prompt = "You're a helpful travel agent. Your task is to find lodging options in a specific destination for a given date range that meet the user's accessibility needs. Ensure that the lodging has the accommodations necessary for the user, if you cannot meet those requirements, then return 'No lodging with accommodations found'."
        )

    def create_booking_step(self):
        return Step(
            name="Create Booking",
            function=create_booking,
            inputs=[self.lodging_info, self.accessibility_needs, self.user_info],
            system_prompt = "Given the lodging information and trip context. Pre-fill the form for the user. Context: {self.lodging_info}; User Information: {self.user_info}."
        )

    def generate_directions_step(self, start_location, end_location, date):
        return Step(
            name="Generate Directions",
            function=generate_directions,
            inputs=[start_location, end_location, self.accessibility_needs, date],
            system_prompt = "Given that the user is going to {end_location} on {date}, generate detailed, accessible directions from {start_location} to the destination. Allow ample time and include step-by-step accessible options for the following needs: {self.accessibility_needs}."
        )

    def generate_overall_plan_step(self, date, itinerary):
        return Step(
            name="Generate Overall Plan",
            function=generate_overall_plan,
            inputs=[self.accessibility_needs, self.destination, [date], self.user_info, itinerary],
            system_prompt = "Given the knowledge of the previous steps, combine the following into a comprehensive, accessible travel plan. Ensure accessibility is prioritized throughout. Be as detailed as possible. Provide each date as a separate response, so that way you can maximize the information provided to the user. Provide the location or lodging website URL. Lodging: {self.find_lodging_step().output}, Booking: {self.create_booking_step().output}, Directions: {self.generate_directions_step(itinerary[0], itinerary[-1], date).output}."
        )

    def create_chain(self):
        lodging_step = self.find_lodging_step()
        self.lodging_info = lodging_step.run()  # Update lodging_info attribute
        
        booking_step = self.create_booking_step()
        booking_output = booking_step.run()
        
        # Generate high-level itineraries for each day
        for date in self.dates:
            itinerary_step = self.generate_itinerary_step(date)
            high_level_itinerary = itinerary_step.run()
            self.high_level_itineraries.append((date, high_level_itinerary))
        
        detailed_itineraries = []
        for date, high_level_itinerary in self.high_level_itineraries:
            # Generate detailed directions for each high-level itinerary
            day_directions = []
            for i in range(len(high_level_itinerary)):
                start_location = self.lodging_info if i == 0 else high_level_itinerary[i-1]
                end_location = high_level_itinerary[i]
                directions_step = self.generate_directions_step(start_location, end_location, date)
                directions_output = directions_step.run()
                day_directions.append(directions_output)
            # Generate overall plan with detailed directions
            overall_plan_step = self.generate_overall_plan_step(date, high_level_itinerary)
            day_plan = overall_plan_step.run()
            detailed_itineraries.append(day_plan)
        
        return detailed_itineraries

In [0]:
class TravelAgent:
    def __init__(self, accessibility_needs, destination, dates, user_info):
        self.accessibility_needs = accessibility_needs
        self.destination = destination
        self.dates = dates
        self.user_info = user_info
        self.lodging_info = None
        self.high_level_itineraries = []

    def generate_itinerary_step(self, date):
        return Step(
            name="Generate Itinerary",
            function=generate_itinerary,
            inputs=[self.accessibility_needs, self.destination, [date]],
            system_prompt = "You are a helpful travel agent. Your task is to generate a high-level travel itinerary for a user with specific accessibility needs for the date {date}. The user may not have specific plans in mind, so prioritize convenience and accessibility while viewing the key attractions of each destination. Include 1-2 activities and nearby meal options. The user is arriving to {self.destination}. Date Context {date}. Accessibility Needs: {self.accessibility_needs}."
        )

    def find_lodging_step(self):
        return Step(
            name="Find Lodging",
            function=find_lodging,
            inputs=[self.accessibility_needs, self.destination, self.dates],
            system_prompt = "You're a helpful travel agent. Your task is to find lodging options in a specific destination for a given date range that meet the user's accessibility needs. Ensure that the lodging has the accommodations necessary for the user, if you cannot meet those requirements, then return 'No lodging with accommodations found'."
        )

    def create_booking_step(self):
        return Step(
            name="Create Booking",
            function=create_booking,
            inputs=[self.lodging_info, self.accessibility_needs, self.user_info],
            system_prompt = "Given the lodging information and trip context. Pre-fill the form for the user. Context: {self.lodging_info}; User Information: {self.user_info}."
        )

    def generate_directions_step(self, start_location, end_location, date):
        return Step(
            name="Generate Directions",
            function=generate_directions,
            inputs=[start_location, end_location, self.accessibility_needs, date],
            system_prompt = "Given that the user is going to {end_location} on {date}, generate detailed, accessible directions from {start_location} to the destination. Allow ample time and include step-by-step accessible options for the following needs: {self.accessibility_needs}."
        )

    def generate_overall_plan_step(self, date, itinerary):
        return Step(
            name="Generate Overall Plan",
            function=generate_overall_plan,
            inputs=[self.accessibility_needs, self.destination, [date], self.user_info, itinerary],
            system_prompt = "Given the knowledge of the previous steps, combine the following into a comprehensive, accessible travel plan. Ensure accessibility is prioritized throughout. Be as detailed as possible. Provide each date as a separate response, so that way you can maximize the information provided to the user. Provide the location or lodging website URL. Lodging: {self.find_lodging_step().output}, Booking: {self.create_booking_step().output}."
        )

    def create_chain(self):
        lodging_step = self.find_lodging_step()
        self.lodging_info = lodging_step.run()  # Update lodging_info attribute
        
        booking_step = self.create_booking_step()
        booking_output = booking_step.run()
        
        # Generate high-level itineraries for each day
        for date in self.dates:
            itinerary_step = self.generate_itinerary_step(date)
            high_level_itinerary = itinerary_step.run()
            self.high_level_itineraries.append((date, high_level_itinerary))
        
        overall_plans = []
        for date, high_level_itinerary in self.high_level_itineraries:
            overall_plan_step = self.generate_overall_plan_step(date, high_level_itinerary)
            day_plan = overall_plan_step.run()
            overall_plans.append(day_plan)
        
        return overall_plans

In [0]:
# Example usage
accessibility_needs = "wheelchair access, visual aids"
destination = "Paris"
dates = ["2025-06-10", "2025-06-12"]
user_info = {"name": "John Doe", "contact": "john.doe@example.com"}

travel_agent = TravelAgent(accessibility_needs, destination, dates, user_info)
travel_chain = travel_agent.create_chain()
display(travel_chain)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-8300164410613014>, line 8
      5 user_info = {"name": "John Doe", "contact": "john.doe@example.com"}
      7 travel_agent = TravelAgent(accessibility_needs, destination, dates, user_info)
----> 8 travel_chain = travel_agent.create_chain()
      9 display(travel_chain)

File <command-4846317734201343>, line 66, in TravelAgent.create_chain(self)
     64 for date, high_level_itinerary in self.high_level_itineraries:
     65     overall_plan_step = self.generate_overall_plan_step(date, high_level_itinerary)
---> 66     day_plan = overall_plan_step.run()
     67     overall_plans.append(day_plan)
     69 return overall_plans

File <command-4846317734201338>, line 12, in Step.run(self)
     10 def run(self):
     11     # Implement the logic to execute the function with inputs
---> 12     self.output = self.function(*self.input